In [12]:
#imports
from rocketcea.cea_obj_w_units import CEA_Obj
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere
import multiprocessing
from rocketisp.geometry import Geometry
from rocketisp.stream_tubes import CoreStream
from rocketisp.efficiencies import Efficiencies
from rocketisp.nozzle.nozzle import Nozzle
from rocketisp.geometry import solidCylVol, solidFrustrumVol
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.interpolate import griddata
import ipywidgets as widgets
from IPython.display import display
import cantera as ct
from PIL import Image
import csv

# Check number of processors
print(f"You have {multiprocessing.cpu_count()} processors")

You have 24 processors


In [95]:
# Initialize CEA Rocket object
rocket = CEA_Obj(oxName='LOX', fuelName='RP-1', temperature_units='degK',
 cstar_units='m/sec', specific_heat_units='kJ/kg degK',
 sonic_velocity_units='m/s', enthalpy_units='J/kg',
 density_units='kg/m^3')

rocket.get_eqratio(MR=2.6)
[MW, MF] = rocket.get_SpeciesMoleFractions(MR=2.6, frozen=1)
MF

{'*CO': [np.float64(0.3187512058749371),
  np.float64(0.3187512058749371),
  np.float64(0.31343405125941476),
  np.float64(0.25741524961437984)],
 '*CO2': [np.float64(0.1343221229432334),
  np.float64(0.1343221229432334),
  np.float64(0.14665268356913358),
  np.float64(0.24930320409420118)],
 '*H': [np.float64(0.04636576183424115),
  np.float64(0.04636576183424115),
  np.float64(0.04145495940345892),
  np.float64(0.0016956520952801928)],
 'HCO': [np.float64(5.6305716368810345e-06),
  np.float64(5.6305716368810345e-06),
  np.float64(3.2560108915794616e-06),
  np.float64(0.0)],
 'HO2': [np.float64(4.60790321544504e-05),
  np.float64(4.60790321544504e-05),
  np.float64(3.0148083578336004e-05),
  np.float64(0.0)],
 '*H2': [np.float64(0.08684782521639382),
  np.float64(0.08684782521639382),
  np.float64(0.08494349384531287),
  np.float64(0.09739285410455127)],
 'H2O': [np.float64(0.29435418526846896),
  np.float64(0.29435418526846896),
  np.float64(0.30979319124616683),
  np.float64(0.39354

In [ ]:
gas = ct.Solution('c12H24_cmb_mech.yaml') # Cantera object for detailed combustion analysis
fuel = 'C12H24:1'  # RP-1
oxidizer = 'O2:14'  # Oxygen
MR = 2.6 # Mixture ratio O/F

gas.set_mixture_fraction(1/(1+MR), fuel, oxidizer) # Set mixture fraction. ct uses mixture fraction as fuel / (fuel + oxidizer)
phi = gas.equivalence_ratio(fuel, oxidizer, basis='mole') # Calculate equivalence ratio
print(f'Equivalence ratio: {phi:.2f}')

Equivalence ratio: 1.32


C:\Users\asing\AppData\Local\Temp\ipykernel_23084\4205598503.py:1: UserWarning: NasaPoly2::validate: 
For species C12H24, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -0.5566423500000255
	Value computed using high-temperature polynomial: -3.5114989489268

  gas = ct.Solution('c12H24_com.yaml') # Cantera object for detailed combustion analysis
C:\Users\asing\AppData\Local\Temp\ipykernel_23084\4205598503.py:1: UserWarning: NasaPoly2::validate: 
For species C12H24, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  95.44412384656303
	Value computed using high-temperature polynomial: 88.67560292843137

  gas = ct.Solution('c12H24_com.yaml') # Cantera object for detailed combustion analysis


In [97]:
# Function to define the mole fraction equations
# X: array of mole fractions (X_C02, X_CO, X_H20)
# phi: equivalence ratio
def mole_fraction_equations(X):
    equation_1 =  X[0] + X[1] + X[2] - 1  # Sum of mole fractions equals 1
    equation_2 = X[2] - X[0] - X[1]  # Ratio of Carbon to Hydrogen
    equation_3 = 4/3 * phi * X[0] + 2/3 * phi * X[1] + 2/3 * phi * X[2] - 2 * X[2]  # Ratio of Hydrogen to Oxygen
    return [equation_1, equation_2, equation_3]

X0 = [1, 1, 1]  # Initial guess for mole fractions
X_solution = fsolve(mole_fraction_equations, X0)
print(f'Mole fractions: CO2 = {X_solution[0] * 100:.3f}%, CO = {X_solution[1]*100:.3f}%, H2O = {X_solution[2]*100:.3f}%')

Mole fractions: CO2 = 13.976%, CO = 36.024%, H2O = 50.000%


In [ ]:
T_O2 = 90 # K
T_C12H24 = 110 # K
P = 5100 * 6894.76 # psia --> Pa
reactants = ct.ThermoPhase('c12H24_cmb_mech.yaml')
O2 = ct.Quantity(reactants, constant='HP')
O2.TPX = T_O2, P, 'O2:1'
O2.moles = 18
C12H24 = ct.Quantity(reactants, constant='HP') # Set Enthalpy Constant
C12H24.TPX = T_C12H24, P, 'C12H24:1'
C12H24.moles = phi 
products = O2 + C12H24
products.equilibrate('HP', solver='gibbs')
print(products.report())



  gas:

       temperature   3955.3 K
          pressure   3.5163e+07 Pa
           density   25.273 kg/m^3
  mean mol. weight   23.637 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -4.8225e+05       -1.1399e+07  J
   internal energy       -1.8736e+06       -4.4285e+07  J
           entropy             10827        2.5591e+05  J/K
    Gibbs function       -4.3305e+07       -1.0236e+09  J
 heat capacity c_p            2063.8             48782  J/K
 heat capacity c_v              1712             40467  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2         0.0068015          0.079745           -18.543
                 H        0.00095306          0.022349           -9.2716
                 O         0.0059632           0.00881           -14.895

C:\Users\asing\AppData\Local\Temp\ipykernel_23084\4080360346.py:4: UserWarning: NasaPoly2::validate: 
For species C12H24, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -0.5566423500000255
	Value computed using high-temperature polynomial: -3.5114989489268

  reactants = ct.ThermoPhase('c12H24_com.yaml')
C:\Users\asing\AppData\Local\Temp\ipykernel_23084\4080360346.py:4: UserWarning: NasaPoly2::validate: 
For species C12H24, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  95.44412384656303
	Value computed using high-temperature polynomial: 88.67560292843137

  reactants = ct.ThermoPhase('c12H24_com.yaml')
